# MNIST Handwritten Digit Problem #2
The objective of this notebook is to combine network design with optimization methods.  We will re-solve the MNIST digit problem with more advanced tools.  You should target an error rate of 2%.

In [ ]:
require 'nn'
require 'paths'
require 'optim'
print('ready')

-- set the number of threads [set to number of cores]
torch.setnumthreads(4)

In [ ]:
-- snippet shamelessly ripped from https://github.com/torch/tutorials/blob/master/A_datasets/mnist.lua 
-- load the training data (downloads when first used)
tar = 'http://torch7.s3-website-us-east-1.amazonaws.com/data/mnist.t7.tgz'

if not paths.dirp('mnist.t7') then
   print('==> downloading dataset')
   os.execute('wget ' .. tar)
   os.execute('tar xvf ' .. paths.basename(tar))
end

train_file = 'mnist.t7/train_32x32.t7'
test_file = 'mnist.t7/test_32x32.t7'

----------------------------------------------------------------------
print('==> loading dataset')

-- We load the dataset from disk (it's straightforward if we have it in the right format t7)
train_data = torch.load(train_file,'ascii')
test_data = torch.load(test_file,'ascii')

In [ ]:
-- print some training data
print(train_data)

In [ ]:
-- example image
itorch.image(train_data.data[{500,1}])

In [ ]:
-- raw data
train_data.data[{500,1}]

In [ ]:
-- let's preprocess the data a bit, so they are between 0 and 1
train_data.data = train_data.data:reshape(60000, 32*32):type('torch.DoubleTensor'):mul(1.0 / 256.0)
test_data.data = test_data.data:reshape(10000, 32*32):type('torch.DoubleTensor'):mul(1.0 / 256.0)

-- remove mean
--train_data.data = train_data.data - torch.repeatTensor(torch.mean(train_data.data,2),1,1024)
--test_data.data = test_data.data - torch.repeatTensor(torch.mean(test_data.data,2),1,1024)

## A new network
- in addition to using the optim package, we also define a new network structure from [Zeiler, 2013](http://www.matthewzeiler.com/pubs/googleTR2012/googleTR2012.pdf)

In [ ]:
-- build our NN model [structure from from Zeiler et al 2013]
net = nn.Sequential()
net:add(nn.Linear(32*32, 500))
net:add(nn.ReLU())
net:add(nn.Linear(500,300))
net:add(nn.ReLU())
net:add(nn.Linear(300,10))
loss = nn.CrossEntropyCriterion()

print(net)

In [ ]:
-- Glorot & Bengio per-layer initialization of network
-- See also Sutskever 2013
local tmp = math.sqrt(1. / net:get(1).bias:size(1))
net:get(1).weight:uniform(-tmp, tmp)
net:get(1).bias:zero() -- per Bengio & Bergstra 2012
tmp = math.sqrt(1. / net:get(3).bias:size(1))
net:get(3).weight:uniform(-tmp, tmp)
net:get(3).bias:zero()
tmp = math.sqrt(1. / net:get(5).bias:size(1))
net:get(5).weight:uniform(-tmp, tmp)
net:get(5).bias:zero()

print('Random initialization')

opt_state = {}

In [ ]:
-- params
nepochs = 10
batch_size = 100
etest = 1
diagnostic_level = 1

-- we optimize with AdaDelta
execute_optimizer = optim.adadelta
opt_config = {
    rho = 0.9,
    eps = 1e-6
}

-- dataset sizes
ntrain = train_data.labels:size(1)
--ntrain = 4000 -- small subset of the dataset to tune the learning rate
ntest = test_data.labels:size(1)
--ntest = 1000

-- retrieve parameters and gradients from the model
parameters,gradParameters = net:getParameters()

-- train the network
for e=1,nepochs do
    
    local timer = torch.Timer() 
    local confusion = optim.ConfusionMatrix(10, {'0','1','2','3','4','5','6','7','8','9'})
    
    for b=1,ntrain/batch_size do
        
        -- build a closure that will evaluate the error & gradient for this minibatch
        local feval = function(x)

            -- get new parameters
            if x ~= parameters then
                parameters:copy(x)
            end

            -- reset gradients
            gradParameters:zero()
            
            local err = 0

            -- evaluate function for complete mini batch
            for i=1,batch_size do
                
                -- select current training example
                local ndx = (b-1) * batch_size + i
                local x = train_data.data[ndx]
                local t = train_data.labels[ndx]

                -- run it through the network & accumulate error
                local y = net:forward(x)
                err = err + loss:forward(y, t)

                -- run it backward from the loss criterion
                local dt_dy = loss:backward(y, t) 
                net:backward(x, dt_dy)

                confusion:add(y, t)
            end

            -- return f and df/dX
            return err,gradParameters
        end
        
        --optim.sgd(feval, parameters, sgd_config, sgd_state)
        execute_optimizer(feval, parameters, opt_config, opt_state)
    end

    print('**** Epoch ' .. e .. ' ****')
    if diagnostic_level > 0 then
        print(confusion:__tostring__())
    end
    
    -- compute testing error every etest epochs
    if e % etest == 0 then
        confusion:zero()
        local terr = 0
        local misses = 0
        for i=1,ntest do
            local x = test_data.data[i]
            local t = test_data.labels[i]
            local y = net:forward(x)
            local _, digit = torch.max(y,1)
            digit = digit[1] - 1
            confusion:add(y, t)
            if digit ~= t then
                misses = misses + 1
            end
            
            terr = terr + loss:forward(y, t)
        end
        print('Testing error after epoch ' .. e .. ' = ' .. terr .. ' misses = ' .. misses)
        if diagnostic_level > 0 or e == nepochs then
            print(confusion:__tostring__())
        end
    end
    
    if diagnostic_level > 1 then
        -- print diagnostic info about the training process
        for _, l in ipairs({1,3,5}) do
            local w = net:get(l).weight
            local nw = w:nElement()
            local w_max =  torch.max(torch.abs(w))
            local wg = net:get(l).gradWeight
            local wg_max =  torch.max(torch.abs(wg))
            local b = net:get(l).bias
            local nb = b:nElement()
            local b_max = torch.max(torch.abs(b))
            print('layer ' .. l)
            print('  weight norms max/2/1 [' .. w_max .. ', ' .. torch.norm(w,2)/nw .. ', ' .. torch.norm(w,1)/nw .. ']')
            print('  w/grad norms max/2/1 [' .. wg_max .. ', ' .. torch.norm(wg,2)/nw .. ', ' .. torch.norm(wg,1)/nw .. ']')
            print('  bias norms max/2/1 [' .. b_max .. ', ' .. torch.norm(b,2)/nb .. ', ' .. torch.norm(b,1)/nb .. ']')
        end
    end    
    print('Epoch took ' .. timer:time().real .. ' seconds.')
end

In [ ]:
torch.save('demo_net.dat', net)

In [ ]:
gnuplot.imagesc(net:get(1).weight, 'color')

In [ ]:
local terr = 0
local p = 0
local ntest = test_data.labels:size(1)
local errlist = {}
for i=1,ntest do
    local x = test_data.data[i]
    local t = test_data.labels[i]
    local y = net:forward(x)
    local logprob, digit = torch.max(y,1)
    if digit[1] ~= t then 
        errlist[#errlist+1]={i, (t-1), (digit-1)}
        p = p + 1
    end
    terr = terr + loss:forward(y, t)
end
--print(errlist)
print('Testing error = ' .. terr .. ' in counts ' .. p .. '/' .. ntest)


In [ ]:
ndx = 126
itorch.image(test_data.data[ndx]:reshape(32,32))
y = net:forward(test_data.data[ndx])
print(torch.cat(torch.range(0,9),y,2))
logprob, digit = torch.max(y,1)
local digit = digit[1] - 1
local target = test_data.labels[ndx] - 1
print('Prediction ' .. digit .. ' real label ' .. target)

In [ ]:
print(test_data.data[ndx]:reshape(32,32))